# Interactive plotting of mean-sea-level pressure for Vladiana cyclone as seen on the native grid and an interpolated r1x1 grid

We here only look at the 40 km run with 2-moment cloud microphysics, which is one of the run in which the identification of the cyclone core pressure is complicated by small-scale pressure variations.

The interpolated data was obtained by cdo remapcon.

## Load python infrastructure

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from ipywidgets import interactive

In [2]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:37662 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 48 Memory: 134.22 GB


## Load ICON MSLP data

Also import the simulation dictionary and the plot color dictionary.

In [3]:
import sys
sys.path.append('/pf/b/b380459/nawdex-hackathon/shared')
import dict_nawdexsims

simdict   = dict_nawdexsims.simdictionary()
colordict = dict_nawdexsims.colordictionary()

Note that we only load data between 2016-09-22T12:00 and 2016-09-24T11:00, as this is the time period in which Vladiana intensified.

In [4]:
def load_mslp_native(sim):
    path  = '/scratch/b/b380459/icon_4_hackathon/'
        
    # grid 
    dict_gridres = {'80km': 'R80000m', '40km': 'R40000m', '20km': 'R20000m',
                    '10km': 'R10000m', '5km': 'R5000m', '2km': 'R2500m'}
    gridres = simdict[sim]['res']
    fname   = path+'/grids/icon-grid_nawdex_78w40e23n80n_'+dict_gridres[gridres]+'.nc'
    ds_grid = xr.open_dataset(fname)[['cell_area','clat', 'clon']].rename({'cell': 'ncells'})
    
    fname   = path+'/'+sim+'/'+sim+'_2016*_2d_30min_DOM01_ML_0*.nc'
    ds_mslp = ( xr.open_mfdataset(fname, combine='by_coords',parallel=True, 
                                  engine='h5netcdf', chunks={'time': 1})[['pres_msl','tot_prec']].
                sel(time=slice('2016-09-22T12:00', '2016-09-24T11:00')))
    
    # merge and return dataset
    return xr.merge([ds_grid, ds_mslp])

def load_mslp_remap(sim):
    #  I am also loading tot_prec, which has the further advantage that this will be a dataset instead of a dataarray
    path  = '/work/bb1018/b380459/NAWDEX/ICON_OUTPUT_NWP/'
    fname = path+'/'+sim+'/post/'+sim+'_2016*_2d_30min_alltimesteps.gridr1x1.nc'
    ds    = ( xr.open_mfdataset(fname, engine='h5netcdf', chunks={'time': 1})[['pres_msl','tot_prec']].
              sel(time=slice('2016-09-22T12:00', '2016-09-24T11:00')))
    
    return ds

We only load the Vladiana simulation for 40 km and 2-moment microphysics. The selection is done by means of start, conv and mphys.

In [5]:
ds_native = []
ds_remap  = []
for sim in list(simdict.keys()):
    # search for the correct simulation
    if simdict[sim]['start']=='20160922' and simdict[sim]['conv']==1 and simdict[sim]['res']=='40km' and simdict[sim]['mphys']==2:
        print('Loading', sim)
        ds_native = load_mslp_native(sim)
        ds_remap  = load_mslp_remap(sim)

Loading nawdexnwp-40km-mis-0002


In [6]:
def plot_field_native(field, time):
    var=ds_native[field].isel(time=time)
    plt.figure(figsize=(12,8))
    plt.tricontourf(np.rad2deg(ds_native.clon), np.rad2deg(ds_native.clat), var, np.linspace(980e2,1000e2,20))
    plt.colorbar()
    plt.xlim(-40,-10); plt.ylim(45,65)
    plt.title(ds_native.isel(time=time).time.values)
    plt.xlabel('longitude'); plt.ylabel('latitude')
    plt.show()
    return

def plot_field_remap(field, time):
    var=ds_remap[field].isel(time=time)
    plt.figure(figsize=(12,8))
    plt.contourf(ds_remap.lon, ds_remap.lat, var, np.linspace(980e2,1000e2,20))
    plt.colorbar()
    plt.xlim(-40,-10); plt.ylim(45,65)
    plt.title(ds_remap.isel(time=time).time.values)
    plt.xlabel('longitude'); plt.ylabel('latitude')
    plt.show()
    return

w = interactive(plot_field_native, field=['pres_msl'], time=(0,len(ds_native.time)-1,1))
#w = interactive(plot_field_remap, field=['pres_msl'], time=(0,len(ds_remap.time)-1,1))


output = w.children[-1]
output.layout.width = '600px'
output.layout.height = '450px'
display(w)

interactive(children=(Dropdown(description='field', options=('pres_msl',), value='pres_msl'), IntSlider(value=…

## Shut down dask

In [7]:
client.close()